<a href="https://colab.research.google.com/github/alex-petrov-git/konspektum/blob/main/vid2md_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Как пользоваться:

1. Загрузи необходимые зависимости (см ячейку ниже)
2. Для работы с YandexGPT (опционально):
 - загрузите файл с промптом под названием prompt.txt (в файлах Google Colab), [пример файла с промптом](https://drive.google.com/file/d/1wXR-T4d4bZUokKqHZrUjbttdyGloxu31/view?usp=drive_link)
 - вставьте свои YC_FOLDER_ID и YC_TOKEN в части 1.4 ноутбука
3. Во вкладке "Среда выполнения" нажми "перезапустить сеанс и выполнить весь код"
4. Дождись пока запустится последняя ячейка
5. Отправляй боту https://t.me/vid2md_bot ссылку на видео ([например](https://drive.google.com/file/d/1PqifWs7x8k-8K2P-T0XkfamGbi6HcUnR/view?usp=drive_link)) с публичным доступом (гугл/яндекс диск).
6. Полученный от бота markdown файл вставляй в obsidian vault

In [ ]:
# if you need to delete smth
#!rm -r slide_images

In [ ]:
# dependencies
#!pip install pyTelegramBotAPI
#!pip install -U openai-whisper
#!sudo apt update
#!sudo apt install tesseract-ocr
#!sudo apt install tesseract-ocr-rus
#!pip install pytesseract
#!pip install moviepy
#!pip install pix2tex
#!pip install yandex-cloud-ml-sdk

In [ ]:
import telebot
import requests
import re
from moviepy.editor import VideoFileClip
import whisper
import os
import time
import cv2
import os
from PIL import Image, ImageEnhance
import pytesseract
import threading
import difflib
import pandas as pd
import time
from pix2tex import cli as pix2tex_cli
from yandex_cloud_ml_sdk import YCloudML

In [ ]:
BOT_TOKEN = '8184444332:AAHxlP0YAmoAAnYouePKWTm0hjhVWEar6vQ'
bot = telebot.TeleBot(BOT_TOKEN)

In [ ]:
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, """\
Привет! Я бот для транскрибации учебных видео в конспекты Obsidian.
Пожалуйста, отправьте мне публичную ссылку на видеофайл Google Диска или Yandex Disk.\
""")

## Main function

In [ ]:
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    bot.reply_to(message, "Начинаю обработку...")
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Получено сообщение от пользователя {message.chat.id}: {message.text}")
    video_filepath = download_video(message.text)

    if video_filepath:
        transcription_result, slides_result, audio_filepath = process_downloaded_video(bot, video_filepath, message)

        if transcription_result and slides_result:
            transcription = transcription_result.get('transcription')
            slide_data = slides_result.get('slide_data', {})
            saved_image_paths = slides_result.get('saved_image_paths', {})

            if transcription is not None:
                bot.reply_to(message, "Обработка видео завершена, генерирую конспект.")
                print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Обработка видео завершена, генерирую конспект.")
                md_content = generate_markdown(saved_image_paths, transcription, video_filepath, slide_data)

                # Генерация конспекта с помощью YandexGPT с использованием промпта из файла
                prompt_file = '/content/prompt.txt'
                summary = generate_summary_with_yandexgpt(md_content, prompt_file)

                if summary:
                    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Конспект сгенерирован с помощью YandexGPT (промпт из файла).")
                    send_markdown_file(bot, message.chat.id, summary) # Отправляем сгенерированный конспект
                else:
                    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Не удалось сгенерировать конспект с помощью YandexGPT. Отправляю базовый Markdown.")
                    send_markdown_file(bot, message.chat.id, md_content) # Отправляем базовый Markdown

                cleanup_files(video_filepath, audio_filepath)
                print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Конспект отправлен пользователю {message.chat.id}.")

            else:
                bot.reply_to(message, "Произошла ошибка при транскрибации аудио.")
                print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при транскрибации аудио для видео: {video_filepath}")
        else:
            bot.reply_to(message, "Произошла ошибка при обработке видео.")
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при обработке видео: {video_filepath}")
    else:
        bot.reply_to(message, "Пожалуйста, отправьте корректную публичную ссылку на видео с Google Диска или Yandex Disk.")
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Пользователь {message.chat.id} отправил некорректную ссылку: {message.text}")


## 1. General processing

### 1.1 Скачивание видео

In [ ]:
def download_video(message_text):
    video_filepath = None
    if 'drive.google.com' in message_text:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Начинаю скачивание файла с Google Диска...")
        video_filepath = download_google_drive_file(message_text)
        if video_filepath:
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Файл с Google Диска успешно скачан: {video_filepath}")
        else:
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Не удалось скачать файл с Google Диска.")
    elif 'yadi.sk' in message_text:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Начинаю скачивание файла с Yandex Disk...")
        video_filepath = download_yandex_disk_file(message_text)
        if video_filepath:
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Файл с Yandex Disk успешно скачан: {video_filepath}")
        else:
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Не удалось скачать файл с Yandex Disk.")
    else:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Получена некорректная ссылка: {message_text}")
    return video_filepath

def download_google_drive_file(url):
    try:
        # Извлекаем ID файла из URL
        file_id = None
        match = re.search(r'id=([^&]+)', url)
        if match:
            file_id = match.group(1)
        else:
            match = re.search(r'/d/([^/]+)/', url)
            if match:
                file_id = match.group(1)

        if file_id:
            gdrive_url = f"https://drive.google.com/uc?id={file_id}&export=download"
            response = requests.get(gdrive_url, stream=True)
            response.raise_for_status()  # Проверяем, что запрос был успешным

            filename = f"gdrive_video_{file_id}.mp4" # Можно сделать более умное определение расширения
            filepath = f"/content/{filename}"
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Начинаю запись файла с Google Диска: {filepath}")
            with open(filepath, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Файл с Google Диска успешно записан: {filepath}")
            return filepath
        else:
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Не удалось извлечь ID файла из URL Google Диска: {url}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при скачивании файла с Google Диска: {e}")
        return None

def download_yandex_disk_file(url):
    try:
        base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download'
        params = {'public_key': url}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        download_url = response.json()['href']
        file_response = requests.get(download_url, stream=True)
        file_response.raise_for_status()

        filename = "yandex_video.mp4" # Опять же, можно улучшить определение расширения
        filepath = f"/content/{filename}"
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Начинаю запись файла с Yandex Disk: {filepath}")
        with open(filepath, "wb") as f:
            for chunk in file_response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Файл с Yandex Disk успешно записан: {filepath}")
        return filepath
    except requests.exceptions.RequestException as e:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при скачивании файла с Yandex Disk: {e}")
        return None
    except KeyError:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Не удалось получить ссылку на скачивание с Yandex Disk.")
        return None

### 1.2 Извлечение информации

In [ ]:
def process_downloaded_video(bot, video_filepath, message):
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Начинаю обработку видео: {video_filepath}")
    audio_filepath = extract_audio(video_filepath)
    if audio_filepath:
        bot.reply_to(message, "Аудио извлечено. Запускаю обработку...")
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Аудио извлечено: {audio_filepath}")
        try:
            clip = VideoFileClip(video_filepath)
            duration_seconds = clip.duration
            clip.close()
            estimated_time_minutes = int(duration_seconds * (10/7) / 60)
            estimated_time_seconds = estimated_time_minutes * 60 if estimated_time_minutes > 0 else 60 # Если оценка 0, ставим хотя бы минуту

            progress_message = bot.send_message(message.chat.id, f"Обработка началась: [ ] 0%")
            progress_bar_length = 20
            start_time = time.time()
            previous_progress_percent = -1 # Инициализируем значением, которое точно будет отличаться

            transcription_result = {}
            slides_result = {}

            audio_thread = threading.Thread(target=transcribe_audio_thread, args=(audio_filepath, transcription_result))
            video_thread = threading.Thread(target=process_video_frames_thread, args=(video_filepath, slides_result))

            audio_thread.start()
            video_thread.start()
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Запущены потоки транскрибации и обработки видео.")

            while audio_thread.is_alive() or video_thread.is_alive():
                elapsed_time = time.time() - start_time
                progress_percent = min(100, int((elapsed_time / (estimated_time_seconds + 30)) * 100)) # Добавляем небольшой запас в 30 секунд

                if progress_percent != previous_progress_percent: # Проверяем, изменился ли прогресс
                    filled_length = int(progress_bar_length * progress_percent // 100)
                    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
                    bot.edit_message_text(f"Обработка: [{bar}] {progress_percent}%", message.chat.id, progress_message.message_id)
                    previous_progress_percent = progress_percent # Обновляем предыдущий процент

                time.sleep(5) # Обновляем прогресс каждые 5 секунд

            # Финальное обновление до 100% (гарантируем, что сообщение обновится при завершении)
            bot.edit_message_text(f"Обработка завершена: [{'█' * progress_bar_length}] 100%", message.chat.id, progress_message.message_id)
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Потоки транскрибации и обработки видео завершены.")

            return transcription_result, slides_result, audio_filepath
        except Exception as e:
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при определении длительности видео: {e}")
            bot.send_message(message.chat.id, "Транскрибация и анализ слайдов запущены. Пожалуйста, подождите.")
            # Если не удалось определить длительность, запускаем потоки без прогресс-бара
            transcription_result = {}
            slides_result = {}
            audio_thread = threading.Thread(target=transcribe_audio_thread, args=(audio_filepath, transcription_result))
            video_thread = threading.Thread(target=process_video_frames_thread, args=(video_filepath, slides_result))
            audio_thread.start()
            video_thread.start()
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Запущены потоки транскрибации и обработки видео (без прогресс-бара).")
            audio_thread.join()
            video_thread.join()
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Потоки транскрибации и обработки видео завершены (без прогресс-бара).")
            return transcription_result, slides_result, audio_filepath
    else:
        bot.reply_to(message, "Произошла ошибка при извлечении аудио из видео.")
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при извлечении аудио из видео: {video_filepath}")
        return None, None, None

### 1.3 Генерация и отправка .md файла

In [ ]:
def generate_markdown(saved_image_paths, transcription, video_filepath, slide_data): # Убрали slide_texts
    md_content = "# Полная транскрибация\n\n"
    md_content += f"{transcription}\n\n"

    md_content += "# Информативные слайды\n\n"
    informative_slides_data = sorted(slide_data.items())
    for timestamp, data in informative_slides_data:
        md_content += f"**Время:** {timestamp}\n\n"
        md_content += f"{data['text']}\n\n"
        if data['formulas']:
            md_content += "## Формулы:\n\n"
            for formula in data['formulas']:
                md_content += f"$${formula}$$\n\n"
        if data['tables']:
            md_content += "## Таблицы:\n\n"
            for table in data['tables']:
                # Здесь нужно будет реализовать форматирование таблицы в Markdown
                md_content += f"```\n{table}\n```\n\n" # Пока просто выводим как есть
        md_content += "---\n\n"

    return md_content

def send_markdown_file(bot, chat_id, md_content):
    with open("output.md", "w") as f:
        f.write(md_content)
    with open("output.md", "rb") as file:
        bot.send_document(chat_id, file)
    os.remove("output.md") # Удаляем временный файл после отправки
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Markdown файл отправлен в чат {chat_id}.")

def cleanup_files(video_filepath, audio_filepath, slide_images_dir='slide_images'):
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Начинаю очистку временных файлов.")
    if os.path.exists(video_filepath):
        os.remove(video_filepath)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Удален видеофайл: {video_filepath}")
    if audio_filepath and os.path.exists(audio_filepath):
        os.remove(audio_filepath)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Удален аудиофайл: {audio_filepath}")
    if os.path.exists(slide_images_dir):
        for filename in os.listdir(slide_images_dir):
            file_path = os.path.join(slide_images_dir, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
            except Exception as e:
                print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при удалении {file_path}: {e}")
        os.rmdir(slide_images_dir)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Удалена директория слайдов: {slide_images_dir}")
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Очистка временных файлов завершена.")

### 1.4 Формирование итогового конспекта с помощью YandexGPT

In [ ]:
def generate_summary_with_yandexgpt(markdown_content, prompt_file_path):
    YC_FOLDER_ID = 'YOUR_FOLDER_ID'
    YC_TOKEN = 'YOUR_TOKEN'

    try:
        with open(prompt_file_path, 'r', encoding='utf-8') as f:
            llm_prompt = f.read()
    except Exception as e:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при чтении файла с промптом: {e}")
        return None

    try:
        sdk = YCloudML(
            folder_id=YC_FOLDER_ID,
            auth=YC_TOKEN,
        )

        model = sdk.models.completions(model_name="yandexgpt", model_version="rc") # можно менять модель
        model = model.configure(temperature=0.2, max_tokens=20000)
        print(llm_prompt)
        result = model.run(llm_prompt + "\n\n" + markdown_content)
        return result.alternatives[0].text
    except Exception as e:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при взаимодействии с YandexGPT API (ML SDK): {e}")
        return None

## 2. Обработка аудио

In [ ]:
def extract_audio(video_path):
    try:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Извлекаю аудио из видео: {video_path}")
        videoclip = VideoFileClip(video_path)
        audio_path = video_path.replace(os.path.splitext(video_path)[1], ".mp3")
        videoclip.audio.write_audiofile(audio_path)
        videoclip.close()
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Аудио успешно извлечено: {audio_path}")
        return audio_path
    except Exception as e:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при извлечении аудио: {e}")
        return None

def transcribe_audio_thread(audio_path, result):
    try:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Запущена транскрибация аудио: {audio_path}")
        model = whisper.load_model("small") # Используем модель "small"
        result['transcription'] = model.transcribe(audio_path)["text"]
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Транскрибация аудио завершена.")
    except Exception as e:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при транскрибации аудио: {e}")
        result['transcription'] = None

## 3. Обработка видео

### 3.1 Извлечение и сохранение информативных кадров

In [ ]:
def extract_frames(video_path, output_dir='frames', interval=5):
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Начинаю извлечение кадров из видео: {video_path} с интервалом {interval} секунд.")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Создана директория для кадров: {output_dir}")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Не удалось открыть видеофайл: {video_path}")
        return []

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / frame_rate
    frames_to_extract = set(range(0, total_frames, int(interval * frame_rate)))
    saved_frames = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count in frames_to_extract:
            frame_filename = os.path.join(output_dir, f"frame_{int(frame_count / frame_rate):03d}.png")
            cv2.imwrite(frame_filename, frame)
            saved_frames.append(frame_filename)

        frame_count += 1

    cap.release()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Извлечение кадров завершено. Сохранено {len(saved_frames)} кадров.")
    return saved_frames

def preprocess_image(image_path):
    try:
        img = Image.open(image_path).convert('L')
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(1.5)
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(1.2)
        return img
    except Exception as e:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при предобработке изображения {image_path}: {e}")
        return None

def extract_informative_frames(video_filepath, similarity_threshold=0.6):
    frames_dir = 'frames'
    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)

    extracted_frames = extract_frames(video_filepath, frames_dir)
    informative_frames = {}
    all_frame_paths = extracted_frames  # Сохраняем пути ко всем кадрам для последующей очистки
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Начинаю анализ кадров на информативность (порог {similarity_threshold}).")

    previous_text = None
    for frame_path in extracted_frames:
        timestamp_sec = int(frame_path.split('_')[-1].split('.')[0])
        timestamp_str = f"{timestamp_sec // 60:02d}:{timestamp_sec % 60:02d}"
        full_text = extract_text_from_frame(frame_path) # Извлекаем текст для сравнения

        if full_text:
            if previous_text is None:
                informative_frames[timestamp_str] = frame_path
                print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Информативный слайд найден (первый): {timestamp_str}")
            else:
                similarity = difflib.SequenceMatcher(None, previous_text, full_text).ratio()
                if similarity < similarity_threshold:
                    informative_frames[timestamp_str] = frame_path
                    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Информативный слайд найден (изменение): {timestamp_str}, схожесть: {similarity:.2f}")

            previous_text = full_text
        else:
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Не удалось извлечь текст из кадра: {frame_path}")

    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Анализ кадров завершен. Найдено {len(informative_frames)} информативных слайдов.")
    return informative_frames, all_frame_paths

def save_informative_frames(informative_frames, output_dir='slide_images'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Создана директория для информативных слайдов: {output_dir}")

    saved_image_paths = {}
    for timestamp, frame_path in informative_frames.items():
        image_filename = os.path.join(output_dir, f"slide_{timestamp.replace(':', '_')}.png")
        os.rename(frame_path, image_filename)
        saved_image_paths[timestamp] = image_filename
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Информативный слайд сохранен как: {image_filename}")
    return saved_image_paths

### 3.2 Извлечение текста из кадров

In [ ]:
def extract_text_from_frame(frame_path, lang='rus'):
    try:
        img = preprocess_image(frame_path)
        if img:
            text = pytesseract.image_to_string(img, lang=lang)
            return text.strip()
        else:
            return ""
    except Exception as e:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при распознавании текста на кадре {frame_path}: {e}")
        return ""

### 3.3 Извлечение формул из кадров

In [ ]:
def contains_math_symbols(text, symbols=['=', '+', '*', '/', 'формула']):
    for symbol in symbols:
        if symbol in text:
            return True
    return False

def extract_formulas(image_path, extracted_text):
    if contains_math_symbols(extracted_text):
        try:
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Обнаружены математические символы. Запускаю pix2tex для: {image_path}")
            output = pix2tex_cli.process(image_path)
            if output:
                print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Формула извлечена: {output}")
                return [output]
            else:
                print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Pix2tex не обнаружил формул.")
                return []
        except Exception as e:
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Ошибка при извлечении формул из {image_path}: {e}")
            return []
    else:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Математические символы не найдены в тексте.")
        return []

### 3.4 General slides processing

In [ ]:
def cleanup_temporary_files(all_frame_paths, frames_dir='frames', slide_images_dir='slide_images'):
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Начинаю удаление временных файлов кадров.")
    for frame_path in all_frame_paths:
        if os.path.exists(frame_path):
            os.remove(frame_path)
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Удален временный файл: {frame_path}")

    if os.path.exists(frames_dir):
        os.rmdir(frames_dir)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Удалена временная директория: {frames_dir}")

def process_video_frames_thread(video_filepath, result):
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Запущен поток обработки видеокадров для: {video_filepath}")
    informative_frames, all_frame_paths = extract_informative_frames(video_filepath) # Получаем только информативные кадры и их пути
    saved_image_paths = save_informative_frames(informative_frames)
    slide_data = {} # Словарь для хранения текста, таблиц и формул для каждого слайда

    for timestamp, image_path in saved_image_paths.items():
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Обрабатываю слайд {timestamp} ({image_path})")
        full_text = extract_text_from_frame(image_path) # Извлекаем текст здесь
        formulas = extract_formulas(image_path, full_text) # Передаем извлеченный текст
        slide_data[timestamp] = {'text': full_text, 'formulas': formulas, 'tables': []} # Таблицы пока пустые
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Завершена обработка слайда {timestamp}.")

    cleanup_temporary_files(all_frame_paths)
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Завершена обработка потока видеокадров.")

    result['slide_data'] = slide_data
    result['saved_image_paths'] = saved_image_paths

## ЗАПУСК БОТА

In [ ]:
bot.polling(none_stop=True)